In [1]:
import os
import sys

sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"


In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit, types, typed
from numba.experimental import jitclass
import numpy as np
from numba.types import int32, float32, ListType
from numba import typeof, typed, njit
from numba.typed import List
from numba.experimental import jitclass
import logging
from nb_quantfreedom.nb_enums import CandleBodyType

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
spec = [("value", int32), ("array", float32[:])]


@jitclass(spec)
class Bag(object):
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)


the_bag_type = typeof(Bag(1))

spec = {}
spec["bag"] = the_bag_type
spec["bag_list"] = ListType(the_bag_type)


@jitclass(spec)
class BagListHolder:
    def __init__(self):
        self.bag = Bag(7)

        self.bag_list = List.empty_list(the_bag_type, 10)
        print("self.bag_list = List.empty_list(the_bag_type,10)", self.bag_list)
        self.bag_list.append(self.bag)
        print("self.bag_list.append(self.bag):", self.bag_list)

    def return_value(self):
        return self.bag_list[0].value


@jitclass()
class Hole(object):
    value: int32
    array: float32[:]

    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)


the_hole_type = typeof(Hole(1))

spec = {}
spec["hole"] = the_hole_type
spec["hole_list"] = ListType(the_hole_type)


@jitclass(spec)
class HoleListHolder:
    hole: Hole

    def __init__(self):
        self.hole = Hole(10)

        self.hole_list = List.empty_list(the_hole_type, 10)
        print("self.hole_list = List.empty_list(the_hole_type,10)", self.hole_list)
        self.hole_list.append(self.hole)
        print("self.hole_list.append(self.hole):", self.hole_list)

    def return_value(self):
        return self.hole_list[0].value


@njit(cache=True)
def CachedBagListHolder(num):
    if num < 5:
        z = BagListHolder()
    else:
        z = HoleListHolder()
    x = z.return_value() + 5


value = CachedBagListHolder(1)
print(value)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mCannot unify instance.jitclass.BagListHolder#162c2fe44f0<bag:instance.jitclass.Bag#162c2fe42b0<value:int32,array:array(float32, 1d, A)>,bag_list:ListType[instance.jitclass.Bag#162c2fe42b0<value:int32,array:array(float32, 1d, A)>]> and instance.jitclass.HoleListHolder#162c3b126e0<hole:instance.jitclass.Hole#162c2fc3f70<value:int32,array:array(float32, 1d, A)>,hole_list:ListType[instance.jitclass.Hole#162c2fc3f70<value:int32,array:array(float32, 1d, A)>]> for 'z.2', defined at C:\Users\User\AppData\Local\Temp\ipykernel_21932\4241684536.py (71)
[1m
File "C:\Users\User\AppData\Local\Temp\ipykernel_21932\4241684536.py", line 71:[0m
[1mdef CachedBagListHolder(num):
    <source elided>
        z = HoleListHolder()
[1m    x = z.return_value() + 5
[0m    [1m^[0m[0m
[0m
[0m[1mDuring: typing of assignment at C:\Users\User\AppData\Local\Temp\ipykernel_21932\4241684536.py (71)[0m
[1m
File "C:\Users\User\App

In [4]:
z

In [5]:
spec = {
    "helper_class_self_num": types.float_,
    "helper_class_self_var": types.float_,
}


@jitclass(spec)
class HelperClass(object):
    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        self.helper_class_self_num = helper_class_self_num
        self.helper_class_self_var = helper_class_self_var

    def calc_num_one(self, helper_num):
        pass


@jitclass
class SubClassOne(object):
    helper_class_inst: HelperClass

    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        self.helper_class_inst = HelperClass(
            helper_class_self_num,
            helper_class_self_var,
        )

    def calc_num_one(self, helper_num):
        return helper_num / self.helper_class_inst.helper_class_self_num

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Internal error at <numba.core.typeinfer.CallConstraint object at 0x00000162C2FE7190>.
[1m[1m[1mtoo many positional arguments[0m
[0m[1mDuring: resolving callee type: jitclass.SubClassOne#162c2fe7d90<helper_class_inst:instance.jitclass.HelperClass#162c2fe6d70<>>[0m
[0m[1mDuring: typing of call at C:\Users\User\AppData\Local\Temp\ipykernel_21932\3325894441.py (3)
[0m
Enable logging at debug level for details.
[1m
File "C:\Users\User\AppData\Local\Temp\ipykernel_21932\3325894441.py", line 3:[0m
[1mdef tester_func():
[1m    calculator = SubClassOne(10, 20)
[0m    [1m^[0m[0m
